In [1]:
import os
import sys
import random
import pickle
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.mixture import GaussianMixture
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay,
    precision_recall_fscore_support,
    roc_auc_score
)

warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
#cluster-label mapping
def match_clusters(pred_labels, true_labels):
    n_clusters = len(np.unique(true_labels))  # Assume the number of clusters is known
    confusion_matrix = np.zeros((n_clusters, n_clusters), dtype=int)

    for i in range(len(true_labels)):
        # Ensure indices are integers by converting them to integers
        row_index = int(true_labels[i])
        col_index = int(pred_labels[i])

        # Access the confusion matrix using the integer indices
        confusion_matrix[row_index, col_index] += 1  # Count occurrences

    # Solve optimal assignment problem
    row_ind, col_ind = linear_sum_assignment(confusion_matrix.max() - confusion_matrix)

    # Create mapping
    cluster_mapping = {col: row for row, col in zip(row_ind, col_ind)}

    # Relabel predicted clusters using the mapping
    matched_pred_labels = np.array([cluster_mapping[label] for label in pred_labels])

    return matched_pred_labels, cluster_mapping


#Generates a random batch of data from the input dataset.
def random_batch(X, y=None, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx]

#Creates a visual progress bar as a string to track the progress of a process.
def progress_bar(iteration, total, size=30):
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)

#Displays the progress bar along with the loss and other metrics in real time.
def print_status_bar(iteration, total, loss, metrics=None, size=30):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)

#Calculates the Euclidean norm (L2 norm) of a tensor along its first axis.
def euclid_norm(x):
    return tf.sqrt(tf.reduce_sum(tf.square(x), axis=1))

def clr(epoch, step_size=10, base_lr=1e-4, max_lr=1e-2):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * np.maximum(0., 1 - x)
    return lr # Changed to return a float instead of a TensorFlow tensor

#Creates a fully connected neural network layer with optional dropout for regularization.
# Estimator Layer class definition
class Estimator(keras.layers.Layer):
    # Estimation network
    def __init__(self, hidden_sizes, activation="elu", kernel_initializer="he_normal", dropout_rate=None, **kwargs):
        super().__init__(**kwargs)
        #Initializes a list of dense (fully connected) layers for the hidden layers of the network.
        self.hidden = [
            keras.layers.Dense(size, activation=activation, kernel_initializer=kernel_initializer,
                               kernel_regularizer=keras.regularizers.l2(0.01))
            for size in hidden_sizes[:-1]
        ]
        #Defines the output layer with softmax activation for classification or probability estimation.
        self.out = keras.layers.Dense(hidden_sizes[-1], activation="softmax",
                                      kernel_initializer=kernel_initializer, kernel_regularizer=keras.regularizers.l2(0.01))
        #Initializes a dropout layer to prevent overfitting if dropout_rate is specified.
        self.dropout_layer = keras.layers.Dropout(rate=dropout_rate)

#Defines the forward pass through the Estimator layer.
    def call(self, z):
        for layer in self.hidden:
            z = layer(z)
            if self.dropout_layer.rate:  # Apply dropout only if a dropout rate is specified
                z = self.dropout_layer(z)
        output = self.out(z)
        return output

#Implements the MPPCA model with a neural network-based responsibility estimation.
class NNMPPCA:
    """Neural network -based Mixture of Probabilistic Principal Component Analyzers.
    """

    MODEL_FILENAME = "MPPCA_model" #Name of the file where the trained model will be saved.
    SCALER_FILENAME = "MPPCA_scaler" #Name of the file where the scaler (used for normalization) will be saved.

    def __init__(self, est_sizes, activation, kernel_initializer, dropout_rate=0.2,
                 n_epochs=1000, batch_size = 128, eta=[1, 0.01, 0.001], learning_rate=0.001,
                 patience=10, normalize=True, random_seed=42):


        inputs = keras.layers.Input(shape=(est_sizes[0],), name="input")
        #inputs = keras.layers.Input(shape=(32,), name="input")

        #Uses the Estimator class to create a network for responsibility (gamma) estimation.
        # Use the Estimator as a layer directly in the model
        gamma = Estimator(hidden_sizes=est_sizes[1:],
                          activation=activation,
                          kernel_initializer=kernel_initializer,
                          dropout_rate=dropout_rate)(inputs)

        #Creates the MPPCA model where inputs are mapped to gamma (responsibilities).
        self.mppca = keras.models.Model(inputs=inputs, outputs=[gamma])

        #Stores the hyperparameters for the model and training.
        self.eta = eta
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.patience = patience
        self.seed = random_seed
        #Keeps track of the parameters (phi, mu, L) at different training epochs.
        self.phi_list = []
        self.mu_list = []
        self.L_list = []


    def parameter_initialize(self, inputs, q=3):

        d = inputs.shape[-1]

        # Compute responsibilities (gamma) for each component
        R = self.mppca(inputs)  # Shape: (n_samples, n_components)

        # Calculate total responsibility (gamma_sum) and mixing coefficients (phi)
        R_sum = tf.reduce_sum(R, axis=0)  # Shape: (n_components,)
        phi = tf.reduce_mean(R, axis=0)  # Shape: (n_components,)

        # Compute the mean (mu) for each component
        mu = tf.einsum('ik,il->kl', R, inputs) / R_sum[:, np.newaxis]  # Shape: (n_components, n_features)

        # Center and weight data using gamma for each component
        x_centered_weighted = tf.sqrt(R[:, :, np.newaxis]) * (inputs[:, np.newaxis, :] - mu[np.newaxis, :, :])
        # Shape: (n_samples, n_components, n_features)

        # Compute covariance matrix (S) for each component
        S = tf.einsum('ikl,ikm->klm', x_centered_weighted, x_centered_weighted) / R_sum[:, np.newaxis, np.newaxis]
        # Shape: (n_components, n_features, n_features)
        S = S + tf.eye(S.shape[-1]) * 1e-6  # Add a small value to the diagonal for regularization
        # Eigenvalue decomposition of S and reorder in descending order
        eig, vec = tf.linalg.eigh(S)
        eig, vec = eig[:, ::-1], vec[:, :, ::-1]

        # Estimate noise variance (sigma2) and select top-q eigenvectors (U)
        sigma2 = tf.reduce_mean(eig[:, q:], axis=1)  # Shape: (n_components,)
        U_q = vec[:, :, :q]  # Top-q eigenvectors (shape: [n_components, n_features, q])
        Lambda_q = tf.linalg.diag(eig[:, :q])  # Diagonal matrix of top-q eigenvalues (shape: [n_components, q, q])

        # Create the (Λ_q - σ^2 * I) term and take the square root
        I_q = tf.eye(q)  # Identity matrix of size q (shape: [q, q])
        adjusted_Lambda_q = tf.sqrt(Lambda_q - sigma2[:, tf.newaxis, tf.newaxis] * I_q)  # Shape: [n_components, q, q]

        # Multiply U_q by the square root term to get W
        W = tf.einsum('ikl,ilm->ikm', U_q, adjusted_Lambda_q)  # Shape: [n_components, n_features, q]

        return sigma2, W


    def custom_loss(self, inputs, q):

        d = inputs.shape[-1]
        W_prev = self.W
        sigma2_prev = self.sigma2

        # Compute responsibilities (gamma) for each component
        R = self.mppca(inputs)  # Shape: (n_samples, n_components)

        # Calculate total responsibility (gamma_sum) and mixing coefficients (phi)
        R_sum = tf.reduce_sum(R, axis=0)  # Shape: (n_components,)
        phi = tf.reduce_mean(R, axis=0)  # Shape: (n_components,)

        # Compute the mean (mu) for each component
        mu = tf.einsum('ik,il->kl', R, inputs) / R_sum[:, np.newaxis]  # Shape: (n_components, n_features)

        # Center and weight data using gamma for each component
        x_centered_weighted = tf.sqrt(R[:, :, np.newaxis]) * (inputs[:, np.newaxis, :] - mu[np.newaxis, :, :])
        # Shape: (n_samples, n_components, n_features)

        # Compute covariance matrix (S) for each component
        S = tf.einsum('ikl,ikm->klm', x_centered_weighted, x_centered_weighted) / R_sum[:, np.newaxis, np.newaxis]
        # Shape: (n_components, n_features, n_features)

        # Compute SW, the product of S and W_prev
        SW_prev = tf.einsum('ikl,ilm->ikm', S, W_prev)  # Shape: (n_components, n_features, q)

        # Create sigma2_prev * I as sigma2I, where I is the identity matrix
        I_q = tf.eye(q)  # Identity matrix of size q (shape: [q, q])
        sigma2_prevI_q = sigma2_prev[:, np.newaxis, np.newaxis] * I_q  # Shape: (n_components, q, q)

        # Compute M and its inverse (Minv)
        M = sigma2_prevI_q + tf.einsum('ikl,ilm->ikm', tf.transpose(W_prev, [0, 2, 1]), W_prev)  # Shape: (n_components, q, q)
        M_inv = tf.linalg.inv(M)  # Shape: (n_components, q, q)

        # Compute Minv * W_prev^T (MinvWT)
        M_invW_prevT = tf.einsum('ikl,ilm->ikm', M_inv, tf.transpose(W_prev, [0, 2, 1]))  # Shape: (n_components, q, n_features)

        # # Small value added to the diagonal to ensure numerical stability
        # min_vals_W = tf.linalg.diag(tf.ones(q, dtype=tf.float32)) * 1e-3  # Shape: (q, q)

        # Compute A, used to calculate the new W
        middle_term = sigma2_prevI_q + tf.einsum('ikl,ilm->ikm', M_invW_prevT, SW_prev)  # Shape: (n_components, q, q)



        # Calculate the updated W (W_new) as SW * Ainv
        W_new = tf.einsum('ikl,ilm->ikm', SW_prev, tf.linalg.inv(middle_term))  # Shape: (n_components, n_features, q)

        # Compute Minv * W_new^T (MinvWnT)
        M_invW_newT = tf.einsum('ikl,ilm->ikm', M_inv, tf.transpose(W_new, [0, 2, 1]))  # Shape: (n_components, q, n_features)

        # Calculate the updated sigma2 (sigma2_new) using the trace
        sigma2_new = 1 / d * tf.linalg.trace(S - tf.einsum('ikl,ilm->ikm', SW_prev, M_invW_newT))


        # model covariance
        I_d = tf.eye(d)  # Identity matrix of size q (shape: [q, q])
        C = ((sigma2_new[:, np.newaxis, np.newaxis]*I_d) +
             tf.einsum('ikl,ilm->ikm', W_new, tf.transpose(W_new, [0, 2, 1])))


        # Cholesky decomposition of covariance matrix with regularization for stability
        min_vals = I_d * 1e-4            # (d, d)
        L = tf.linalg.cholesky(C + min_vals[np.newaxis, :, :])  # L: (n_components, d, d)

        # Center input data by subtracting component means
        x_centered = inputs[:, np.newaxis, :] - mu[np.newaxis, :, :]  # (n_samples, n_components, d)

        # Solve triangular system for Mahalanobis distance
        v = tf.linalg.triangular_solve(L, tf.transpose(x_centered, [1, 2, 0]))  # (n_components, d, n_samples)

        # Compute log-determinant of covariance from Cholesky factor
        log_det_cov = 2.0 * tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L)), axis=1)  # (n_components,)

        # Calculate log-probabilities (logits) for each component
        logits = tf.math.log(phi[:, np.newaxis]) - 0.5 * (
            tf.reduce_sum(tf.square(v), axis=1)
            + d * tf.math.log(2.0 * tf.constant(np.pi, dtype="float32"))
            + log_det_cov[:, np.newaxis]
        )  # logits: (n_components, n_samples)

        # Compute negative log-likelihood using log-sum-exp trick
        reconstruction_probability = tf.reduce_logsumexp(logits, axis=0)  # (n_samples,)


        # Calculate the reconstruction error of MMPCA

        # Compute transition matrix and individual component reconstruction in fewer steps
        B = tf.einsum('ikl, ilm->ikm', W_new, tf.linalg.inv(tf.einsum('ikl, ilm->ikm', tf.transpose(W_new, [0, 2, 1]), W_new))) \
            @ tf.transpose(W_new, [0, 2, 1])  # Shape: (n_components, d, d)

        # Calculate individual component reconstruction of x and overall reconstruction
        x_k = tf.einsum('ikl, ilm->ikm', B, tf.transpose(x_centered, [1, 2, 0])) + mu[:, :, tf.newaxis]
        x_res = tf.einsum('ik, ikm->im', R, tf.transpose(x_k, [2, 0, 1]))  # Shape: (n_samples, d)

        # Compute reconstruction error
        reconstruction_error = tf.reduce_sum(tf.square(inputs - x_res), axis=1)


       # Extract individual weights (eta_1, eta_2, eta_3) for different loss components
        eta_1, eta_2, eta_3 = self.eta[0], self.eta[1], self.eta[2]

        # Calculate the penalty term as the sum of the inverses of the diagonal elements of middle_term
        penalty_term = tf.reduce_sum(tf.divide(1, tf.linalg.diag_part(middle_term)))

        # Compute the total loss as a weighted combination of three components:
        # 1. Negative reconstruction probability (multiplied by eta_1)
        # 2. Reconstruction error (multiplied by eta_2)
        # 3. Penalty term (multiplied by eta_3)

        total_loss = (- eta_1 * tf.reduce_mean(reconstruction_probability)
                      + eta_2 * tf.reduce_mean(reconstruction_error) + eta_3 * penalty_term)

        # Update the model's parameters with the latest values
        self.phi = phi             # Mixing coefficient
        self.mu = mu               # Mean of each component
        self.W = W_new             # Updated transformation matrix
        self.sigma2 = sigma2_new   # Updated variance for noise
        self.L = L                 # Cholesky factor of the covariance

        # Return the computed total loss
        return total_loss

    def fit(self, inputs, X_test=None, y_test=None):
        # Set random seeds for reproducibility
        tf.random.set_seed(self.seed)
        np.random.seed(seed=self.seed)

        # Split the input data into training and validation sets
        X_train, X_valid = train_test_split(inputs, test_size=0.3, random_state=42)

        # Determine the number of steps per epoch based on the batch size
        n_steps = len(X_train) // self.batch_size
        # Initialize the optimizer with the specified learning rate
        optimizer = keras.optimizers.Nadam(learning_rate=self.learning_rate)

        # Initialize metrics to track training and validation loss
        mean_loss = keras.metrics.Mean(name='mean_loss')
        metrics = keras.metrics.Mean(name='val_loss')
        minimum_val_loss = float("inf")  # Keep track of the lowest validation loss
        best_epoch = 1                   # Track the best epoch
        wait = 0                          # Early stopping counter

        # Dimensionality for the latent space
        q = 3

        # Initialize parameters sigma2 and W based on the training data
        self.sigma2, self.W = self.parameter_initialize(X_train, q)

        # Start the training loop for the specified number of epochs
        for epoch in range(1, self.n_epochs + 1):
            print("Epoch {}/{}".format(epoch, self.n_epochs))

            # Reset loss metrics at the start of each epoch
            mean_loss.reset_state()
            metrics.reset_state()

            # Iterate over each batch of training data
            for step in range(1, n_steps + 1):
                # Select a random batch from the training data
                X_batch = random_batch(X_train, batch_size=self.batch_size)

                # Compute the loss and gradients for the batch
                with tf.GradientTape() as tape:
                    main_loss = self.custom_loss(X_batch, q)  # Custom loss for the batch
                    loss = tf.add_n([main_loss] + self.mppca.losses)  # Add any additional losses
                gradients = tape.gradient(loss, self.mppca.trainable_variables)  # Calculate gradients
                optimizer.apply_gradients(zip(gradients, self.mppca.trainable_variables))  # Apply gradients

                # Ensure any constrained variables remain within their constraints
                for variable in self.mppca.variables:
                    if variable.constraint is not None:
                        variable.assign(variable.constraint(variable))

            # Calculate training and validation loss at the end of the epoch
            train_loss = tf.add_n([self.custom_loss(X_train, q)] + self.mppca.losses)
            mean_loss(train_loss)  # Update mean training loss
            val_loss = tf.add_n([self.custom_loss(X_valid, q)] + self.mppca.losses)
            metrics(val_loss)  # Update mean validation loss
            print_status_bar(len(inputs), len(inputs), mean_loss, [metrics])



    def predict(self, X_test):
        # Get the dimensionality of the input data
        d = X_test.shape[-1]

        # Retrieve model parameters at the best epoch for prediction
        phi = self.phi_list[self.best_epoch - 1]  # Mixing coefficients for each component
        mu = self.mu_list[self.best_epoch - 1]    # Means of each component
        L = self.L_list[self.best_epoch - 1]      # Cholesky factor of the covariance matrices

        # Center the test data by subtracting the mean of each component
        x_centered = X_test[:, np.newaxis, :] - mu[np.newaxis, :, :]  # Shape: (n_samples, n_components, d)

        # Solve for the Mahalanobis distance using triangular solve with the Cholesky factor
        v = tf.linalg.triangular_solve(L, tf.transpose(x_centered, [1, 2, 0]))  # Shape: (n_components, d, n_samples)

        # Calculate the log-determinant of the covariance matrix using the Cholesky factor
        log_det_cov = 2.0 * tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L)), axis=1)  # Shape: (n_components,)

        # Compute log-probabilities (logits) for each component using the log-sum-exp trick
        logits = tf.math.log(phi[:, np.newaxis]) - 0.5 * (
            tf.reduce_sum(tf.square(v), axis=1)  # Mahalanobis distance term
            + d * tf.math.log(2.0 * tf.constant(np.pi, dtype="float32"))  # Normalization constant
            + log_det_cov[:, np.newaxis]  # Log-determinant term
        )  # Shape: (n_components, n_samples)

        # Use log-sum-exp to compute the reconstruction probabilities for each sample
        reconstruction_probabilities = tf.reduce_logsumexp(logits, axis=0)  # Shape: (n_samples,)

        # Return the negative reconstruction probabilities as the "energy" score
        return -reconstruction_probabilities.numpy()


## Semi MPPCA

In [ ]:
n_classes = 3
train,test = train_test_split(df, test_size=0.3,stratify=df['labels'], random_state=None)

# Desired split: 100 instances per class for the first dataset
instances_per_class = 10
# Initialize lists to store the sampled and remaining data
sampled_data = []
remaining_data = []
# Perform sampling for each class
for class_label in range(n_classes):
    # Subset for the current class
    class_subset = train[train['labels'] == class_label]
    # Sample the required number of instances
    sampled_subset = class_subset.sample(n=instances_per_class, random_state=None)
    sampled_data.append(sampled_subset)
    # Identify remaining instances
    remaining_subset = class_subset.drop(sampled_subset.index)
    remaining_data.append(remaining_subset)

# Concatenate sampled subsets to form the first dataset
dataset_1 = pd.concat(sampled_data)
# Concatenate remaining subsets to form the second dataset
dataset_2 = pd.concat(remaining_data)
# Print the results
print(f"Dataset 1 shape: {dataset_1.shape}")  # Should have 200 instances (100 per class)
print(f"Dataset 2 shape: {dataset_2.shape}")  # Remaining instances

# Reset indices for clarity
train_labeled = dataset_1.reset_index(drop=True)
train_unlabeled = dataset_2.reset_index(drop=True)

y_train_labeled = train_labeled.labels
X_train_labeled= train_labeled.drop([ 'labels'], axis= 1)
X_train_labeled = X_train_labeled.to_numpy()
y_train_labeled = y_train_labeled.to_numpy()

y_train_unlabeled = train_unlabeled.labels
X_train_unlabeled= train_unlabeled.drop([ 'labels'], axis= 1)
X_train_unlabeled = X_train_unlabeled.to_numpy()
y_train_unlabeled = y_train_unlabeled.to_numpy()

X_test = test.drop(['labels'], axis= 1)
y_test = test.labels
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

scaler = MinMaxScaler().fit(X_train_unlabeled)
# Transform both the training and test sets using the fitted scaler
X_train_unlabeled, X_train_labeled = scaler.transform(X_train_unlabeled), scaler.transform(X_train_labeled)
X_test = scaler.transform(X_test)

# Define and initialize the MPPCA model with specific architecture and training parameters
nn_mppca = NNMPPCA(est_sizes=[32, 50, 15, 3],
                   activation="selu",
                   kernel_initializer="glorot_normal",
                   dropout_rate=0.2,
                   n_epochs=10,
                   batch_size=256,
                   eta=[1, 0.5, 0.005],
                   learning_rate=0.002,
                   patience=20)

# Fit the MPPCA model to the training data and validate using the test data
nn_mppca.fit(X_train_unlabeled)

# Creating a copy
new_model = tf.keras.models.clone_model(nn_mppca.mppca)
new_model.set_weights(nn_mppca.mppca.get_weights())

# Freeze the input and hidden layers only
for layer in new_model.layers[:-1]:  # Skip the last layer (output layer)
    layer.trainable = False

# The output layer remains trainable
new_model.layers[-1].trainable = True


true_labels = y_train_labeled
#print("true_labels:", true_labels)
pred_labels = tf.argmax(new_model(X_train_labeled), axis=-1)
#print("pred_labels:", pred_labels.numpy())

matched_pred_labels, cluster_mapping = match_clusters(pred_labels.numpy(), true_labels)
#print("matched_pred_labels:", matched_pred_labels)
#print("cluster_mapping:", cluster_mapping)

# Extract keys and values as separate lists
predicted_classes = list(cluster_mapping.keys())
true_classes = list(cluster_mapping.values())
#print(f"Cluster Mapping: {cluster_mapping}, predicted_classes: {predicted_classes}")

original_list = predicted_classes.copy()
#print("original_list:", original_list)

accuracy = accuracy_score(true_labels, matched_pred_labels)
#print("Accuracy_1:", accuracy)

if predicted_classes == list(range(n_classes)):
    print("Dont need to swap weights")
else:
    print("Need to swap weights")
     # Get all the weights of the last layer (estimator_2)
    all_weights = new_model.layers[-1].get_weights()
    # Assuming the weights you want to modify are at index 4 (last_layer_weights)
    last_layer_weights = all_weights[4]
    # Swapping columns according to the order
    swapped_weights = tf.gather(last_layer_weights, predicted_classes, axis=1)
     # Replace the modified weights back into the list
    all_weights[4] = swapped_weights
    # Update the layer with the complete list of weights
    new_model.layers[-1].set_weights(all_weights)


# Get model predictions
y_pred = new_model.predict(X_train_labeled)
# Convert predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the class with the highest probability
# Compute accuracy
accuracy = accuracy_score(true_labels, y_pred_labels)
print("Accuracy_2:", accuracy)

# Define the learning rate scheduler
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # Monitors training loss
    factor=0.5,      # Reduces learning rate by 50%
    patience=10,      # Waits 3 epochs for improvement before reducing learning rate
    min_lr=0.002      # Stops reducing once learning rate reaches this value
)

#Define the unsupervised early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Track self-supervised loss
    patience=10,
    restore_best_weights=True,
    mode='min'
)

# Split Train/Validation - take 10 samples for val (where len(X_train_labeled) > 10)
test_size = 10 / len(X_train_labeled)

X_train_labeled, X_val, y_train_labeled, y_val = train_test_split(
    X_train_labeled, y_train_labeled, test_size=test_size, stratify=y_train_labeled, random_state=None
)

# #Retrain model / finetune with few labeled samples
# Compile the new model
new_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.05),
                  loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

# Train the model with the learning rate scheduler
history = new_model.fit(X_train_labeled, y_train_labeled,
                validation_data=(X_val, y_val),  # Provide test data for validation
                epochs=150,
                batch_size=10,
                callbacks=[reduce_lr, early_stopping]
                        )


# Predict on the test data
y_pred = new_model.predict(X_test)

# Get the predicted classes
y_pred_flag = np.argmax(y_pred, axis=1)


# Compute metrics
accuracy = accuracy_score(y_test, y_pred_flag)
# Calculate precision, recall, and F1-score using predicted class labels
precision = precision_score(y_test, y_pred_flag, average='weighted', zero_division=0)  # Changed to y_pred_flag
f1 = f1_score(y_test, y_pred_flag, average='weighted')  # Changed to y_pred_flag
recall = recall_score(y_test, y_pred_flag, average='weighted')  # Changed to y_pred_flag

y_test = y_test.astype(int)
# Convert y_true to one-hot encoding
y_true_onehot = np.zeros((len(y_test), len(set(y_test))))
y_true_onehot[np.arange(len(y_test)), y_test] = 1
# Compute AUC score
auc = roc_auc_score(y_true_onehot, y_pred, multi_class="ovr")


#Pint the results
print(f"Average Accuracy: {accuracy: .4f}" )
print(f"Average Precision: {precision: .4f}")
print(f"Average F1 Score: {f1: .4f}")
print(f"Average Recall: {recall : .4f}")
print(f"Average AUC score:{auc: .4f}")